California - doesn't have attribute "Breed" and "Age"

*   California "Age" = "Intake.Date" - "DOB"

*   "Breed" hasn't been check yet, but removed for now.

Age column is in years.


In [11]:
import pandas as pd

austin = pd.read_csv("merged_austin_data.csv")
california = pd.read_csv("california-animal-data.csv")
indiana = pd.read_csv("indiana-animal-data.csv")

#adding "state" attribute - identify which state each pet came from
austin["State"] = "Austin"
california["State"] = "California"
indiana["State"] = "Indiana"


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
#rename, drop irrelevant columns
austin = austin.rename(columns={
    'Animal.ID.x': 'Animal.ID',
    'Name.x': 'Name',
    'MonthYear.x': 'MonthYear.Intake',
    'MonthYear.y': 'MonthYear.Outcome',
    'Animal.Type.x': 'Animal.Type',
    'Breed.x': 'Breed',
    'Color.x': 'Color',
    'DateTimeUTC.x': 'Intake.Date',
    'DateTimeUTC.y': 'Outcome.Date',
    'Sex.upon.Outcome': 'Sex',
    'Age.upon.Outcome': 'Age'

})

austin = austin.drop(columns=[col for col in austin.columns if '.y' in col])
austin = austin.drop(columns=['Date.of.Birth', 'Intake.Condition', 'DateTime.x', 'Outcome.Subtype', 'MonthYear.Intake', 'MonthYear.Outcome', 'Sex.upon.Intake', 'Age.upon.Intake', 'Found.Location'])

print(austin.columns)

california = california.rename(columns={
    'Animal ID': 'Animal.ID',
    'Animal Name': 'Name',
    'Animal Type': 'Animal.Type',
    'Primary Color': 'Color',
    'Animal.Type.x': 'Animal.Type',
    'Intake Type': 'Intake.Type',
    'Intake Date': 'Intake.Date',
    'Outcome Type': 'Outcome.Type',
    'Outcome Date': 'Outcome.Date',
    'Date.of.Birth': 'DOB'
})

austin = austin.dropna()

california = california.drop(columns=['Secondary Color', 'Intake Subtype', 'Reason for Intake',
                         'Crossing', 'Jurisdiction', 'Outcome Subtype',
                         'latitude', 'longitude', 'intake_is_dead',
                         'outcome_is_dead', 'was_outcome_alive', 'geopoint', 'Intake Condition'], errors='ignore')

california = california.dropna()

#calculate age = intake take - DOB
#convert DOB, Intake.Date from object into datetime data type
california['DOB'] = pd.to_datetime(california['DOB'], errors='coerce')
california['Intake.Date'] = pd.to_datetime(california['Intake.Date'], errors='coerce')

#calculate age in years
california['Age'] = (california['Intake.Date'] - california['DOB']).dt.days / 365.25

#round age to 2d.p.
california['Age'] = california['Age'].round(2)

#drop DOB
california = california.drop(columns=['DOB'], errors='ignore')

print(california.columns)

indiana = indiana.rename(columns={
    'id': 'Animal.ID',
    'animalname': 'Name',
    'speciesname': 'Animal.Type',
    'basecolour': 'Color',
    'breedname': 'Breed',
    'intakedate': 'Intake.Date',
    'intakereason': 'Intake.Type',
    'animalage': 'Age',
    'sexname': 'Sex',
    'movementdate': 'Outcome.Date',
    'movementtype': 'Outcome.Type'
})

indiana = indiana.drop(columns=['sheltercode', 'identichipnumber', 'istransfer', 'istrial', 'returndate', 'returnedreason', 'deceaseddate', 'deceasedreason', 'diedoffshelter', 'puttosleep',
       'isdoa','location'], errors='ignore')

indiana = indiana.dropna()

print(indiana.columns)

#dimensions of individual data sets
print(austin.shape)
print(california.shape)
print(indiana.shape)


Index(['Animal.ID', 'Name', 'Intake.Type', 'Animal.Type', 'Breed', 'Color',
       'Intake.Date', 'Outcome.Type', 'Sex', 'Age', 'Outcome.Date', 'State'],
      dtype='object')
Index(['Animal.ID', 'Name', 'Animal.Type', 'Color', 'Sex', 'Intake.Date',
       'Intake.Type', 'Outcome.Date', 'Outcome.Type', 'State', 'Age'],
      dtype='object')
Index(['Animal.ID', 'Intake.Date', 'Intake.Type', 'Name', 'Breed', 'Color',
       'Animal.Type', 'Age', 'Sex', 'Outcome.Date', 'Outcome.Type', 'State'],
      dtype='object')
(117917, 12)
(16047, 11)
(10243, 12)


In [14]:
common_columns = list(set(austin.columns) & set(indiana.columns))
common_columns

['Breed',
 'Sex',
 'Animal.ID',
 'Intake.Type',
 'Name',
 'Animal.Type',
 'Age',
 'Outcome.Date',
 'Outcome.Type',
 'Color',
 'State',
 'Intake.Date']

In [15]:
#concatenate 3 datasets
# common_columns = list(set(austin.columns) & set(california.columns) & set(indiana.columns))

austin_aligned = austin.reindex(columns=common_columns)
# california_aligned = california.reindex(columns=common_columns)
indiana_aligned = indiana.reindex(columns=common_columns)

optipaw = pd.concat([austin_aligned,indiana_aligned], ignore_index=True)

print(optipaw.shape)
print(optipaw.head())

(128160, 11)
             Sex Animal.ID      Intake.Type Animal.Type    Name  \
0  Neutered Male   A006100    Public Assist         Dog   Scamp   
1  Neutered Male   A006100    Public Assist         Dog   Scamp   
2  Neutered Male   A047759  Owner Surrender         Dog    Oreo   
3  Neutered Male   A134067    Public Assist         Dog  Bandit   
4  Spayed Female   A141142            Stray         Dog  Bettie   

      Outcome.Type         Outcome.Date   State         Color       Age  \
0  Return to Owner  2014-12-20 16:35:00  Austin  Yellow/White   7 years   
1  Return to Owner  2014-03-08 17:10:00  Austin  Yellow/White   6 years   
2         Transfer  2014-04-07 15:12:00  Austin      Tricolor  10 years   
3  Return to Owner  2013-11-16 11:54:00  Austin   Brown/White  16 years   
4  Return to Owner  2013-11-17 11:40:00  Austin   Black/White  15 years   

           Intake.Date  
0  2014-12-19 10:21:00  
1  2014-03-07 14:26:00  
2  2014-04-02 15:55:00  
3  2013-11-16 09:02:00  
4  2013-

In [16]:
#remove duplicate rows
print(optipaw.duplicated().sum())
optipaw = optipaw.drop_duplicates()

#remove rows with missing values
print(optipaw.isnull().sum())
optipaw = optipaw.dropna() #should be 0, NA rows removed when individual data sets were dealt

#check outcome dates are later than intake dates, otherwise remove row
optipaw['Outcome.Date'] = pd.to_datetime(optipaw['Outcome.Date'], errors='coerce')
optipaw['Intake.Date'] = pd.to_datetime(optipaw['Intake.Date'], errors='coerce')

optipaw_valid_rows = optipaw[optipaw['Outcome.Date'] > optipaw['Intake.Date']]
num_valid_rows = optipaw_valid_rows.shape[0]
print(num_valid_rows)

#reorder heading
order = ['Animal.ID', 'Name', 'Animal.Type', 'Sex', 'Color', 'Age', 'Breed','Intake.Type', 'Outcome.Type', 'Intake.Date', 'Outcome.Date', 'State']
optipaw_final = optipaw_valid_rows[order]

#change age to unit years
import re

def convert_age_to_years(age_string):
    unit_conversions = {
        'year': 1,
        'month': 1/12,
        'week': 1/52,
        'day': 1/365
    }

    pattern = r'(\d+)\s*(year|month|week|day)s?'
    match = re.match(pattern, age_string.lower())

    if match:
        number = int(match.group(1))
        unit = match.group(2)
        return number * unit_conversions[unit]
    else:
        return None

optipaw_final['Age'] = optipaw_final['Age'].apply(convert_age_to_years)
optipaw_final['Age'] = optipaw_final['Age'].round(2)

print(optipaw_final.head())

optipaw_final.to_csv("/content/optipaw_new_without_CA.csv", index=False)

72
Sex             0
Animal.ID       0
Intake.Type     0
Animal.Type     0
Name            0
Outcome.Type    0
Outcome.Date    0
State           0
Color           0
Age             0
Intake.Date     0
dtype: int64
124914
  Animal.ID    Name Animal.Type            Sex         Color   Age  \
0   A006100   Scamp         Dog  Neutered Male  Yellow/White   7.0   
1   A006100   Scamp         Dog  Neutered Male  Yellow/White   6.0   
2   A047759    Oreo         Dog  Neutered Male      Tricolor  10.0   
3   A134067  Bandit         Dog  Neutered Male   Brown/White  16.0   
4   A141142  Bettie         Dog  Spayed Female   Black/White  15.0   

       Intake.Type     Outcome.Type         Intake.Date        Outcome.Date  \
0    Public Assist  Return to Owner 2014-12-19 10:21:00 2014-12-20 16:35:00   
1    Public Assist  Return to Owner 2014-03-07 14:26:00 2014-03-08 17:10:00   
2  Owner Surrender         Transfer 2014-04-02 15:55:00 2014-04-07 15:12:00   
3    Public Assist  Return to Owner 2013-1